# Convert Apple Health Sleep Data to JSON

In [34]:
import pandas as pd
import json

# read file
df = pd.read_csv('../../data/Apple_Watch_Sleep.csv', sep=';')

df.head()

,type,sourcename,sourceversion,creationdate,startdate,enddate,value,HKTimeZone
0,HKCategoryTypeIdentifierSleepAnalysis,Zepp Life,202108031059,2021-08-26 20:15:20 -0400,2021-08-26 12:29:00 -0400,2021-08-26 12:51:59 -0400,HKCategoryValueSleepAnalysisAsleepUnspecified,NaN
1,HKCategoryTypeIdentifierSleepAnalysis,Zepp Life,202108031059,2021-08-26 20:15:20 -0400,2021-08-26 12:52:00 -0400,2021-08-26 13:07:59 -0400,HKCategoryValueSleepAnalysisAsleepUnspecified,NaN
2,HKCategoryTypeIdentifierSleepAnalysis,Zepp Life,202108031059,2021-08-26 20:15:20 -0400,2021-08-26 13:08:00 -0400,2021-08-26 15:28:59 -0400,HKCategoryValueSleepAnalysisAsleepUnspecified,NaN
3,HKCategoryTypeIdentifierSleepAnalysis,Zepp Life,202108031059,2021-08-26 20:15:20 -0400,2021-08-26 15:29:00 -0400,2021-08-26 15:51:59 -0400,HKCategoryValueSleepAnalysisAsleepUnspecified,NaN
4,HKCategoryTypeIdentifierSleepAnalysis,Zepp Life,202108031059,2021-08-26 20:15:20 -0400,2021-08-26 15:52:00 -0400,2021-08-26 16:21:59 -0400,HKCategoryValueSleepAnalysisAsleepUnspecified,NaN


In [49]:
# Choose time range
df = df[(df['startdate'] >= '2023-10-18') & (df['startdate'] <= '2024-4-12')]
df = df.reset_index(drop=True)

# Sort by startdate
df = df.sort_values(by='startdate')
df = df.reset_index(drop=True) # VERY IMPORTANT IF YOU ARE GOING TO USE THE INDEX

# Group records
df['startdate'] = pd.to_datetime(df['startdate'])
df['min'] = df['startdate'].dt.minute
df['hour'] = df['startdate'].dt.hour
df['day'] = df['startdate'].dt.day
df['month'] = df['startdate'].dt.month
df['year'] = df['startdate'].dt.year
df['creationdate'] = df['creationdate'].astype(str)
df['startdate'] = df['startdate'].astype(str)
df['enddate'] = df['enddate'].astype(str)

# Drop sourcename='GarField iPhone 13'
# df = df[df['sourcename'] != 'GarField iPhone 13']

df.head()

,type,sourcename,sourceversion,creationdate,startdate,enddate,value,HKTimeZone,min,hour,day,month,year
0,HKCategoryTypeIdentifierSleepAnalysis,GarField iPhone 13,17.0,2023-10-18 07:57:08 -0400,2023-10-18 00:22:54-04:00,2023-10-18 00:26:47 -0400,HKCategoryValueSleepAnalysisInBed,America/New_York,22,0,18,10,2023
1,HKCategoryTypeIdentifierSleepAnalysis,GarField iPhone 13,17.0,2023-10-18 07:57:08 -0400,2023-10-18 00:50:38-04:00,2023-10-18 06:43:56 -0400,HKCategoryValueSleepAnalysisInBed,America/New_York,50,0,18,10,2023
2,HKCategoryTypeIdentifierSleepAnalysis,GarField’s Apple Watch,10.0.1,2023-10-18 07:57:08 -0400,2023-10-18 00:55:54-04:00,2023-10-18 01:14:24 -0400,HKCategoryValueSleepAnalysisAsleepCore,America/New_York,55,0,18,10,2023
3,HKCategoryTypeIdentifierSleepAnalysis,GarField’s Apple Watch,10.0.1,2023-10-18 07:57:08 -0400,2023-10-18 00:55:54-04:00,2023-10-18 01:51:24 -0400,HKCategoryValueSleepAnalysisInBed,America/New_York,55,0,18,10,2023
4,HKCategoryTypeIdentifierSleepAnalysis,GarField’s Apple Watch,10.0.1,2023-10-18 07:57:08 -0400,2023-10-18 01:14:24-04:00,2023-10-18 01:51:24 -0400,HKCategoryValueSleepAnalysisAsleepDeep,America/New_York,14,1,18,10,2023


In [51]:
# Save to json
sleepCollection = []
oneDay = []
for i in range(len(df)):
    if i>0 and df['hour'][i-1] < 11 and df['hour'][i] >= 22:
        sleepCollection.append(oneDay) # previous day
        oneDay = [] # new day
    oneRecord = df.iloc[i].to_dict()
    oneDay.append(oneRecord)


with open('../../data/Apple_Watch_Sleep.json', 'w') as f:
    json.dump(sleepCollection, f)